In [54]:
%run "qboot.py"

In [55]:
L = 6
H = sum(1j* maj(i) @ maj(i+1) for i in range(L))
H

i χ0 χ1 +i χ1 χ2 +i χ2 χ3 +i χ3 χ4 +i χ4 χ5 +i χ5 χ6

In [56]:
K = H.krylov(show_progress=True)

    terms weight
 0:     1    0.0
 1:     6    2.0
 2:    10    4.0
 3:    14    5.5
 4:    16    4.0
 5:    16    2.1
 6:    17    4.0
 7:    14    2.4


In [65]:
a = maj(1,2) @ maj(3,4)
with limit(len, 2):
    b = maj(1,2) @ maj(3,4)
a, b

(χ1 χ2 χ3 χ4, 0)

In [66]:
maj(1,2) @ maj(3,4)

χ1 χ2 χ3 χ4

In [37]:
K

OperatorSpace([I,
               0.408i χ0 χ1 +0.408i χ1 χ2 +0.408i χ2 χ3 +0.408i χ3 χ4 +0.408i χ4 χ5 +0.408i χ5 χ6,
               -0.316 χ0 χ1 χ2 χ3 -0.316 χ0 χ1 χ3 χ4 -0.316 χ0 χ1 χ4 χ5 -0.316 χ0 χ1 χ5 χ6 -0.316 χ1 χ2 χ3 χ4 -0.316 χ1 χ2 χ4 χ5 -0.316 χ1 χ2 χ5 χ6 -0.316 χ2 χ3 χ4 χ5 -0.316 χ2 χ3 χ5 χ6 -0.316 χ3 χ4 χ5 χ6,
               -0.144i χ2 χ3 -0.144i χ3 χ4 -0.144i χ4 χ5 +0.289i χ5 χ6 +0.433i χ0 χ3 +0.289i χ0 χ1 +0.433i χ1 χ4 -0.144i χ1 χ2 +0.433i χ2 χ5 +0.433i χ3 χ6,
               -0.258 χ0 χ1 χ2 χ3 +0.0645 χ0 χ1 χ3 χ4 +0.0645 χ0 χ1 χ4 χ5 -0.258 χ0 χ1 χ5 χ6 -0.323 χ0 χ1 χ2 χ5 -0.323 χ0 χ1 χ3 χ6 +0.0645 χ1 χ2 χ3 χ4 +0.387 χ1 χ2 χ4 χ5 +0.0645 χ1 χ2 χ5 χ6 -0.323 χ1 χ2 χ3 χ6 +0.0645 χ2 χ3 χ4 χ5 +0.0645 χ2 χ3 χ5 χ6 -0.258 χ3 χ4 χ5 χ6 -0.323 χ0 χ3 χ4 χ5 -0.323 χ0 χ3 χ5 χ6 -0.323 χ1 χ4 χ5 χ6,
               0.25i χ2 χ3 +0.25i χ3 χ4 -0.25i χ4 χ5 -0.25i χ2 χ5 +0.25i χ3 χ6 +0.25i χ0 χ3 +0.5i χ0 χ5 -0.25i χ1 χ4 +0.5i χ1 χ6 -0.25i χ1 χ2,
               -0.144 χ0 χ1 χ2 χ3 -0.289 χ0 χ1 χ3 χ4